In [1]:
#carregando as bibliotecas

#numpy é uma biblioteca Python usada para computação numérica.Ele fornece suporte para arrays e matrizes grandes e multidimensionais, bem como uma grande coleção de funções matemáticas para operar nesses arrays.
import numpy as np
#torch é uma biblioteca Python usada para construir e treinar redes neurais. Ele fornece suporte para cálculos de tensor, diferenciação automática e aceleração de GPU.
import torch
#glob é um módulo Python usado para encontrar todos os nomes de caminho correspondentes a um padrão especificado de acordo com as regras usadas pelo shell Unix.
import glob
#torch.nn é um módulo na biblioteca do torch que fornece suporte para a construção de redes neurais. Ele fornece um conjunto de camadas e funções pré-definidas que podem ser usadas para construir uma rede neural.
import torch.nn as nn
#torchvision.tranforms é um módulo na biblioteca do torchvision que fornece suporte para transformações de imagens. Ele fornece um conjunto de funções predefinidas que podem ser usadas para transformar imagens, como redimensionar, recortar e inverter.
from torchvision.transforms import transforms
#torch.utils.data é um módulo na biblioteca do torch que fornece suporte para carregamento e processamento de dados. Ele fornece um conjunto de classes e funções que podem ser usadas para criar carregadores de dados, que podem ser usados ​​para carregar dados em lotes durante o treinamento.
from torch.utils.data import DataLoader
#torch.optim é um módulo na biblioteca do torch que fornece suporte para algoritmos de otimização. Ele fornece um conjunto de algoritmos de otimização predefinidos, como descida de gradiente estocástico (SGD) e Adam, que podem ser usados ​​para otimizar os parâmetros de uma rede neural.
from torch.optim import Adam
#torch.autograd é um módulo na biblioteca do torch que fornece suporte para diferenciação automática. Ele permite que os gradientes sejam calculados automaticamente para qualquer operação de tensor.
from torch.autograd import Variable
#O código também importa a biblioteca torchivision, que é uma biblioteca separada que fornece suporte para tarefas de visão computacional, como classificação de imagens e detecção de objetos.
import torchvision
#pathlib é um módulo Python usado para trabalhar com caminhos de arquivos. Ele fornece uma interface orientada a objetos para trabalhar com caminhos de arquivo, facilitando a manipulação de caminhos de arquivo de maneira independente de plataforma.
import pathlib

In [18]:
#checando se possui GPU disponivel
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
#exibe GPU ou CPU
print(device)

cpu


In [20]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])
])

- A função transforms.Compose é uma função utilitária fornecida pela biblioteca PyTorch que permite aos usuários combinar várias transformações de imagem em um único pipeline de transformação. A função recebe uma lista de funções de transformação como entrada e retorna uma nova função que aplica cada transformação na lista em sequência.

- No código fornecido, a função transforms.Compose é usada para criar um pipeline de transformação que consiste em quatro transformações: transforms.Resize, transforms.RandomHorizontalFlip, transforms.ToTensor e transforms.Normalize.

- transforms.Resize((150,150)): Esta transformação redimensiona a imagem de entrada para um tamanho de 150x150 pixels. Isso é feito para garantir que todas as imagens no conjunto de dados tenham o mesmo tamanho, o que é um requisito para treinar modelos de aprendizado profundo.

- transforms.RandomHorizontalFlip(): Esta transformação inverte aleatoriamente a imagem de entrada horizontalmente com uma probabilidade de 0,5. Isso é feito para aumentar o conjunto de dados e aumentar seu tamanho, o que pode melhorar o desempenho do modelo.

- transforms.ToTensor(): Esta transformação converte a imagem de entrada de uma matriz NumPy com valores no intervalo [0, 255] em um tensor PyTorch com valores no intervalo [0, 1]. Isso é feito para garantir que os dados de entrada estejam no formato correto para treinar modelos de aprendizado profundo.

- transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5]): Esta transformação normaliza o tensor de entrada subtraindo a média e dividindo pelo desvio padrão. Os valores de média e desvio padrão usados ​​nesta transformação são [0,5, 0,5, 0,5] e [0,5, 0,5, 0,5], respectivamente. Isso é feito para garantir que os dados de entrada tenham uma média de 0 e um desvio padrão de 1, o que pode melhorar o desempenho do modelo.

- Finalmente, a função transforms.Compose retorna uma nova função que aplica cada transformação na lista em sequência. Essa nova função pode ser usada para transformar imagens de entrada antes de alimentá-las em um modelo de aprendizado profundo.

In [21]:
#Carregando os dados

#Path for training and testing directory
train_path='/home/lbarcat/Documentos/Code/galaxy_detection/galaxy_train/galaxy_train'
test_path='/home/lbarcat/Documentos/Code/galaxy_detection/galaxy_test/galaxy_test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [22]:
#categorias
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [23]:
print(classes)

['anulares', 'barradas', 'elipticas', 'espirais', 'irregulares']


In [24]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=5):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [25]:
model=ConvNet(num_classes=5).to(device)

In [26]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [31]:
num_epochs=60

In [32]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [33]:
print(train_count,test_count)

76 76


In [34]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(0.0002) Train Accuracy: 1.0263157894736843 Test Accuracy: 1.0263157894736843
Epoch: 1 Train Loss: tensor(6.6138e-05) Train Accuracy: 1.0263157894736843 Test Accuracy: 1.0263157894736843
Epoch: 2 Train Loss: tensor(3.0103e-05) Train Accuracy: 1.0263157894736843 Test Accuracy: 1.0263157894736843
Epoch: 3 Train Loss: tensor(6.8009e-05) Train Accuracy: 1.0263157894736843 Test Accuracy: 1.0263157894736843
Epoch: 4 Train Loss: tensor(7.7159e-05) Train Accuracy: 1.0263157894736843 Test Accuracy: 1.0263157894736843
Epoch: 5 Train Loss: tensor(5.8337e-05) Train Accuracy: 1.0263157894736843 Test Accuracy: 1.0263157894736843
Epoch: 6 Train Loss: tensor(3.7410e-05) Train Accuracy: 1.0263157894736843 Test Accuracy: 1.0263157894736843
Epoch: 7 Train Loss: tensor(6.1153e-06) Train Accuracy: 1.0263157894736843 Test Accuracy: 1.0263157894736843
Epoch: 8 Train Loss: tensor(0.0006) Train Accuracy: 1.0263157894736843 Test Accuracy: 1.0263157894736843
Epoch: 9 Train Loss: tensor